In [1]:
! pip install tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.5 MB/s eta 0:00:00


In [11]:
import os
import math
import time
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import tiktoken
import numpy as np
from torch.utils.data import Dataset, DataLoader


In [3]:
@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layer: int = 12 # number of layers
    n_head: int = 12 # number of heads
    n_embd: int = 768 # embedding dimension

In [4]:
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True) # flash attention
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu    = nn.GELU(approximate='tanh')
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing scheme
        self.transformer.wte.weight = self.lm_head.weight

        # init params
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        # idx is of shape (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        # forward the token and posisition embeddings
        pos = torch.arange(0,T , dtype=torch.long, device=idx.device) # shape (T)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embd)
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (B, T, n_embd)
        x = tok_emb + pos_emb
        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
        # forward the final layernorm and the classifier
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        # print(logits)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, device_type):
        # start with all of the candidate parameters (that require grad)
        param_dict = {pn: p for pn, p in self.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == "cuda"
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer

In [5]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")

# added after video, pytorch can be serious about it's device vs. device_type distinction
device_type = "cuda" if device.startswith("cuda") else "cpu"

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

enc = tiktoken.get_encoding("gpt2")

total_batch_size = 524288 # 2**19, ~0.5M, in number of tokens
B = 64 # micro batch size
T = 1024 # sequence length
ddp_rank = 0
ddp_world_size = 1
master_process = True
grad_accum_steps = total_batch_size // (B * T * ddp_world_size)
torch.set_float32_matmul_precision('high')

# create model
model = GPT(GPTConfig(vocab_size=50304))
# model = GPT.from_pretrained("gpt2") # or init from OpenAI GPT-2
model.to(device)
use_compile = False # torch.compile interferes with HellaSwag eval and Generation. TODO fix
if use_compile:
    model = torch.compile(model)
raw_model = model # always contains the "raw" unwrapped model

max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 715
max_steps = 19073 # 19,073 steps is ~1 epoch, if data is 10B tokens and batch size 0.5M tokens
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_steps:
        return max_lr * (it+1) / warmup_steps
    # 2) if it > lr_decay_iters, return min learning rate
    if it > max_steps:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff starts at 1 and goes to 0
    return min_lr + coeff * (max_lr - min_lr)

# optimize!
optimizer = raw_model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device_type=device_type)

# create the log directory we will write checkpoints to and log to
log_dir = "log"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f"log.txt")
with open(log_file, "w") as f: # open for writing to clear the file
    pass

using device: cuda


In [6]:
with open('shahname_fa.txt', 'r', encoding='utf-8') as file:
    text = file.read()

tokensq = enc.encode(text)

with open('tokens.txt', 'w') as file:
    file.write(' '.join(map(str, tokensq)))

print(f"Tokenized {len(tokensq)} tokens.")

Tokenized 3067442 tokens.


In [7]:
from torch.utils.data import Dataset, DataLoader

class ShahnamehDataset(Dataset):
    def __init__(self, file_path, seq_length):
        """
        Custom Dataset for Shahnameh token sequences.

        Args:
            file_path (str): Path to the file containing the tokenized text.
            seq_length (int): Length of each sequence to be used for training.
        """
        self.seq_length = seq_length

        # Read the tokens from the file and convert to integers
        with open(file_path, 'r') as file:
            tokens = list(map(int, file.read().split()))

        # Trim tokens to ensure they can be evenly divided into sequences
        num_sequences = len(tokens) // seq_length
        self.tokens = tokens[:num_sequences * seq_length]

        # Convert tokens list to a PyTorch tensor
        self.tokens_tensor = torch.tensor(self.tokens, dtype=torch.long)

    def __len__(self):
        """
        Returns the total number of sequences in the dataset.
        """
        return len(self.tokens_tensor) - self.seq_length

    def __getitem__(self, idx):
        """
        Returns a single sequence and its target.

        Args:
            idx (int): Index of the sequence.

        Returns:
            tuple: (input sequence, target sequence)
        """
        input_seq = self.tokens_tensor[idx:idx + self.seq_length]
        target_seq = self.tokens_tensor[idx + 1:idx + self.seq_length + 1]
        return input_seq, target_seq

    def next_batch(self, batch_size):
        """
        Creates a DataLoader to yield batches of sequences.

        Args:
            batch_size (int): Number of sequences per batch.

        Returns:
            DataLoader: DataLoader object to iterate over batches.
        """
        return DataLoader(self, batch_size=batch_size, shuffle=True)

In [8]:
file_path = 'tokens.txt'
seq_length = 1024

dloader = ShahnamehDataset(file_path, seq_length)

batch_size = 5
train_loader = dloader.next_batch(batch_size)

In [9]:
max_steps = 100

for step in range(max_steps):
    t0 = time.time()
    last_step = (step == max_steps )

    # once in a while generate from the model (except step 0, which is noise)
    if ((step > 0 and step % 250 == 0) or last_step) and (not use_compile):
        model.eval()
        num_return_sequences = 4
        max_length = 32
        tokens = enc.encode("سلام من شاهنامه هستم") # interesting start for shahnameh model :))
        tokens = torch.tensor(tokens, dtype=torch.long)
        tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
        xgen = tokens.to(device)
        sample_rng = torch.Generator(device=device)
        sample_rng.manual_seed(42)
        while xgen.size(1) < max_length:
            # forward the model to get the logits
            with torch.no_grad():
                with torch.autocast(device_type=device_type, dtype=torch.bfloat16):
                    logits, loss = model(xgen) # (B, T, vocab_size)
                # take the logits at the last position
                logits = logits[:, -1, :] # (B, vocab_size)
                # get the probabilities
                probs = F.softmax(logits, dim=-1)
                # do top-k sampling of 50 (huggingface pipeline default)
                # topk_probs here becomes (5, 50), topk_indices is (5, 50)
                topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
                # select a token from the top-k probabilities
                # note: multinomial does not demand the input to sum to 1
                ix = torch.multinomial(topk_probs, 1, generator=sample_rng) # (B, 1)
                # gather the corresponding indices
                xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
                # append to the sequence
                xgen = torch.cat((xgen, xcol), dim=1)
        # print the generated text
        for i in range(num_return_sequences):
            tokens = xgen[i, :max_length].tolist()
            decoded = enc.decode(tokens)
            print(f"rank {ddp_rank} sample {i}: {decoded}")

    # do one step of the optimization
    model.train()
    optimizer.zero_grad()
    loss_accum = 0.0
    for micro_step in range(grad_accum_steps):
        # added after video, this field is also used by the forward pass.
        batch_idx, (X_batch, y_batch) = next(enumerate(train_loader))
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        with torch.autocast(device_type=device_type, dtype=torch.bfloat16):
            logits, loss = model(X_batch, y_batch)
        # we have to scale the loss to account for gradient accumulation,
        # because the gradients just add on each successive backward().
        # addition of gradients corresponds to a SUM in the objective, but
        # instead of a SUM we want MEAN. Scale the loss here so it comes out right
        loss = loss / grad_accum_steps
        loss_accum += loss.detach()
        loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    # determine and set the learning rate for this iteration
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    optimizer.step()
    if device_type == "cuda":
        torch.cuda.synchronize() # wait for the GPU to finish work
    t1 = time.time()
    dt = t1 - t0 # time difference in seconds
    batch_size = 64
    # tokens_processed = train_loader.B * train_loader.T * grad_accum_steps * ddp_world_size
    tokens_processed = batch_size * seq_length * grad_accum_steps * ddp_world_size
    tokens_per_sec = tokens_processed / dt
    if loss_accum.item() < 0.0078:
        print("we reached desirable loss === ", loss_accum.item())
    if master_process:
        print(f"step {step:5d} | loss: {loss_accum.item():.6f} | lr {lr:.4e} | norm: {norm:.4f} | dt: {dt*1000:.2f}ms | tok/sec: {tokens_per_sec:.2f}")
        with open(log_file, "a") as f:
            f.write(f"{step} train {loss_accum.item():.6f}\n")


step     0 | loss: 11.046612 | lr 8.3916e-07 | norm: 41.0961 | dt: 15799.79ms | tok/sec: 33183.23
step     1 | loss: 10.901855 | lr 1.6783e-06 | norm: 40.7992 | dt: 14052.13ms | tok/sec: 37310.21
step     2 | loss: 10.643498 | lr 2.5175e-06 | norm: 39.4163 | dt: 14313.78ms | tok/sec: 36628.19
step     3 | loss: 10.286939 | lr 3.3566e-06 | norm: 34.1556 | dt: 14604.46ms | tok/sec: 35899.16
step     4 | loss: 9.904593 | lr 4.1958e-06 | norm: 27.5740 | dt: 14885.33ms | tok/sec: 35221.79
step     5 | loss: 9.533874 | lr 5.0350e-06 | norm: 22.4075 | dt: 15111.78ms | tok/sec: 34693.99
step     6 | loss: 9.230017 | lr 5.8741e-06 | norm: 18.5954 | dt: 15410.68ms | tok/sec: 34021.08
step     7 | loss: 8.952619 | lr 6.7133e-06 | norm: 16.0559 | dt: 15802.56ms | tok/sec: 33177.40
step     8 | loss: 8.688454 | lr 7.5524e-06 | norm: 13.9756 | dt: 15610.62ms | tok/sec: 33585.35
step     9 | loss: 8.469163 | lr 8.3916e-06 | norm: 12.2618 | dt: 15496.85ms | tok/sec: 33831.90
step    10 | loss: 8.28022

In [10]:
tokens = enc.encode("از من چه بر آید:")
tokens = torch.tensor(tokens, dtype=torch.long)
num_return_sequences = 5

tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)


max_length = 40
xgen = tokens.to(device)
while xgen.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        with torch.autocast(device_type=device_type, dtype=torch.bfloat16):
            logits, loss = model(xgen) # (B, T, vocab_size)
        # take the logits at the last position
        last_logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(last_logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        xgen = torch.cat((xgen, xcol), dim=1)
# print the generated text
for i in range(num_return_sequences):
    tokenss = xgen[i, :max_length].tolist()
    decoded = enc.decode(tokenss)
    print(f"rank {ddp_rank} sample {i}: {decoded}")

rank 0 sample 0: از من چه بر آید: با
|چین
|بدهد توده ا
rank 0 sample 1: از من چه بر آید:
|به برپای ناز ب
|ه 
rank 0 sample 2: از من چه بر آید:پشیان پهمی بخ به آ
rank 0 sample 3: از من چه بر آید:یوا رار بایشمر نراهر
rank 0 sample 4: از من چه بر آید:پر ند
|بد
|بدل بهرش ا
